In [2]:
import os
import tensorflow as tf	
import numpy as np
import time
from tf_utils import input_fn_from_dataset,input_fn_frame_from_dataset,save_tf_record,prob_positive_class_from_prediction
from get_data import get_videos_from_folder,get_target_from_csv
from utils import save_solution
# Install the plaidml backend
import plaidml.keras
plaidml.keras.install_backend()
import keras
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D,
                                        MaxPooling2D)
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger

In [3]:
train_folder = os.path.join("../train/")
test_folder = os.path.join("../test/")

train_target = os.path.join('../train_target.csv')
my_solution_file = os.path.join('../solution.csv')

x_train = get_videos_from_folder(train_folder)
y_train = get_target_from_csv(train_target)

In [4]:
# maximum number of frames used as input
pframe = 100 #max([len(x) for x in x_train])
X = np.zeros((x_train.shape[0], pframe, 100, 100,1))
for i in range(x_train.shape[0]):
    if x_train[i].shape[0] >= pframe:
        X[i,: pframe,:x_train[i].shape[1],:x_train[i].shape[2],0] = x_train[i][: pframe,:x_train[i].shape[1],:x_train[i].shape[2]]
    else:
        # repeated padding
        frames, height, width = x_train[i].shape[0], x_train[i].shape[1], x_train[i].shape[2]
        pos = 0
        while pos + frames <= pframe:
            X[i,pos:(pos+frames),:height,:width,0] = x_train[i][:frames,:height,:width]
            pos += frames
        if pos < pframe:
            X[i,pos:pframe,:height,:width,0] = x_train[i][:(pframe-pos),:height,:width]

In [8]:
nb_epoch=50
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (7, 7), strides=(2, 2),activation='relu',padding='same'), input_shape=( pframe, 100, 100,1)))
model.add(TimeDistributed(Conv2D(32, (3,3),
kernel_initializer="he_normal", activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(64, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(128, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(256, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(256, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(512, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(512, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(Dropout(0.5))
model.add(LSTM(256, return_sequences=False, dropout=0.5))
model.add(Dense(2, activation='softmax'))
metrics = ['accuracy']
optimizer = Adam(lr=1e-3, decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer,
                       metrics=metrics)
model.fit(X, y_train, batch_size=32, verbose=1, epochs=nb_epoch)




NotImplementedError: ArgMax is not directly implemented.

In [7]:
x_test = get_videos_from_folder(test_folder)

In [8]:
X_test = np.zeros((x_test.shape[0], 212, 100, 100,1))
for i in range(x_test.shape[0]):
    X_test[i,:x_test[i].shape[0],:x_test[i].shape[1],:x_test[i].shape[2],0] = x_test[i]

In [9]:
! pip install pandas
train_predictions = model.predict(X_test) 


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


NameError: name 'pred' is not defined

In [10]:
solution = prob_positive_class_from_prediction(train_predictions)
save_solution("submission.csv",solution)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices